# Test out the pick em game before moving to its own .exe or .py

#### 2024-12-11

In [1]:
import json
import pandas as pd
import os
import random

## Load data

In [2]:
path_moves_per_pokemon = os.path.join("..", "..", "Scraping", "data", "moves_per_pokemon_orig_names.json")
path_pokemon_main_stats = os.path.join("..", "..", "Scraping", "data", "pokemon_main_stats_data.csv")
path_pokemon_moves_master = os.path.join("..", "..", "Scraping", "data", "pokemon_moves_data.csv")
path_type_effectiveness = os.path.join("..", "data", "type_effectiveness.json")

In [3]:
# Load moves per pokemon as dict
with open(path_moves_per_pokemon, 'r') as f:
    dict_moves_per_pokemon = json.load(f)

# Load type effectiveness as a dict
with open(path_type_effectiveness, 'r') as f:
    dict_type_effectiveness = json.load(f)

In [4]:
# Load dfs
## Each pokemon and their stats
df_pokemon_main_stats = pd.read_csv(path_pokemon_main_stats)

# Pokemon moves
df_pokemon_moves_master = pd.read_csv(path_pokemon_moves_master)
df_pokemon_moves_master['Accuracy'] = df_pokemon_moves_master['Accuracy'].replace('∞', '100').astype(float) # Replace '∞' with 100.0 and convert the column to floats


## Random Selection

### Randomly select two pokemon without replacement and four of their moves without replacement

In [5]:
# Randomly select two Pokémon without replacement
selected_pokemon = random.sample(list(dict_moves_per_pokemon.keys()), 2)

In [6]:
# Select 4 random moves for each Pokémon
battle_pokemon = {
    pokemon: random.sample(dict_moves_per_pokemon[pokemon], 4)
    for pokemon in selected_pokemon
}

In [7]:
battle_pokemon

{'Oranguru': ['After You', 'Gravity', 'Ally Switch', 'Shadow Ball'],
 'Kingambit': ['Guillotine', 'Thief', 'Low Kick', 'Slash']}

## Pokemon battle simulation

In [16]:
def pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness):
    # Initialize Pokemon with their stats (same as before)
    pokemon = {}
    for name, moves in battle_pokemon.items():
        stats = df_pokemon_main_stats[df_pokemon_main_stats['Name'] == name].iloc[0]
        pokemon[name] = {
            'hp': stats['HP'],
            'max_hp': stats['HP'],
            'attack': stats['Attack'],
            'defense': stats['Defense'],
            'sp_atk': stats['Sp. Atk'],
            'sp_def': stats['Sp. Def'],
            'speed': stats['Speed'],
            'type': stats['Type'],
            'moves': moves
        }
    
    # Determine order based on Speed
    order = sorted(pokemon.keys(), key=lambda x: pokemon[x]['speed'], reverse=True)
    
    def calculate_damage(attacker, defender, move_data, effectiveness):
        power = move_data['Power'] if not pd.isna(move_data['Power']) else 0
        
        # Determine if it's a critical hit (1/24 chance)
        is_critical = random.random() < 1/24
        critical_multiplier = 2 if is_critical else 1
        
        if move_data['Category'] == 'Physical':
            damage = (((2 * 50 / 5 + 2) * power * attacker['attack'] / defender['defense']) / 50 + 2) * effectiveness * critical_multiplier
        elif move_data['Category'] == 'Special':
            damage = (((2 * 50 / 5 + 2) * power * attacker['sp_atk'] / defender['sp_def']) / 50 + 2) * effectiveness * critical_multiplier
        else:
            damage = 0
        
        damage = max(1, int(damage))  # Ensure at least 1 damage
        return damage, is_critical

    # Battle loop
    while all(pokemon[p]['hp'] > 0 for p in pokemon):
        for attacker_name in order:
            if pokemon[attacker_name]['hp'] <= 0:
                continue
            
            defender_name = [p for p in pokemon if p != attacker_name][0]
            
            # Select a random move
            move = random.choice(pokemon[attacker_name]['moves'])
            move_data = df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == move].iloc[0]
            
            print(f"\n{attacker_name} used {move} on {defender_name}!")
            
            # Check if the move hits based on accuracy
            accuracy = move_data['Accuracy']
            if pd.isna(accuracy) or random.random() * 100 < accuracy:
                # Move hits
                # Calculate type effectiveness
                effectiveness = 1
                for att_type in pokemon[attacker_name]['type']:
                    for def_type in pokemon[defender_name]['type']:
                        if att_type in dict_type_effectiveness and def_type in dict_type_effectiveness[att_type]:
                            effectiveness *= dict_type_effectiveness[att_type][def_type]
                
                # Calculate damage
                damage, is_critical = calculate_damage(pokemon[attacker_name], pokemon[defender_name], move_data, effectiveness)
                
                # Apply damage
                pokemon[defender_name]['hp'] = max(0, pokemon[defender_name]['hp'] - damage)
                
                # Print battle text
                if is_critical:
                    print("It's a critical hit!")
                if effectiveness > 1:
                    print("It's super effective!")
                elif effectiveness < 1 and effectiveness > 0:
                    print("It's not very effective...")
                elif effectiveness == 0:
                    print("It doesn't affect " + defender_name + "...")
                
                print(f"It dealt {damage} damage. {defender_name}'s HP: {pokemon[defender_name]['hp']}/{pokemon[defender_name]['max_hp']}")
                
                if pokemon[defender_name]['hp'] <= 0:
                    print(f"{defender_name} fainted! {attacker_name} wins!")
                    return attacker_name
            else:
                # Move misses
                print(f"But it missed!")
    
    return "Draw"  # This should never happen in a 1v1 battle, but just in case

In [17]:
pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness)


Oranguru used Ally Switch on Kingambit!
It dealt 1 damage. Kingambit's HP: 99/100

Kingambit used Thief on Oranguru!
It dealt 46 damage. Oranguru's HP: 44/90

Oranguru used After You on Kingambit!
It dealt 1 damage. Kingambit's HP: 98/100

Kingambit used Low Kick on Oranguru!
It dealt 2 damage. Oranguru's HP: 42/90

Oranguru used Ally Switch on Kingambit!
It dealt 1 damage. Kingambit's HP: 97/100

Kingambit used Guillotine on Oranguru!
It dealt 2 damage. Oranguru's HP: 40/90

Oranguru used Gravity on Kingambit!
It dealt 1 damage. Kingambit's HP: 96/100

Kingambit used Thief on Oranguru!
It dealt 46 damage. Oranguru's HP: 0/90
Oranguru fainted! Kingambit wins!


'Kingambit'

In [ ]:
## Now work on:
# PP
# Effects
# Status effects
# Simplified? damage calc - what are we missing?
"""
Note that this is a simplified version and doesn't include all Pokémon battle mechanics 
 (such as status effects, critical hits, or move PP). 
 You can expand on this base to add more complex mechanics as needed.
"""

# 12/12/2024:
# Added in crit chance 
# Added in accuracy
# Added in text for crit, super/not very effective when applicable

In [25]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Self-Destruct']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
703,Self-Destruct,Normal,Physical,200.0,100.0,5.0,User faints.,NaN


In [12]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Guillotine']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
357,Guillotine,Normal,Physical,NaN,30.0,5.0,"One-Hit-KO, if it hits.",NaN


In [18]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Thief']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
849,Thief,Dark,Physical,60.0,100.0,25.0,Also steals opponent's held item.,NaN
